In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
validation_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
train_dataset = CIFAR10(root="./data", train=True, transform=train_transform, download=True)
validation_dataset = CIFAR10(root="./data", train=False, transform=validation_transform, download=True)
print(train_dataset)
print("length of train data", len(train_dataset), "length of validation data", len(validation_dataset), "nr of classes", len(train_dataset.classes))

cifar_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
cifar_validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
length of train data 50000 length of validation data 10000 nr of classes 10


In [3]:
from custommodel import CustomModel

cnn_model = CustomModel(32, len(train_dataset.classes))

In [6]:
from trainer import Trainer
ModelTrainer = Trainer(cnn_model, cifar_train_loader, cifar_validation_loader, 10, 0.001)
ModelTrainer.train()

Epoch [1/10], Step [100/1563], Loss: 2.1426
Epoch [1/10], Step [200/1563], Loss: 1.9223
Epoch [1/10], Step [300/1563], Loss: 1.7796
Epoch [1/10], Step [400/1563], Loss: 1.6943
Epoch [1/10], Step [500/1563], Loss: 1.6279
Epoch [1/10], Step [600/1563], Loss: 1.6276
Epoch [1/10], Step [700/1563], Loss: 1.5591
Epoch [1/10], Step [800/1563], Loss: 1.5394
Epoch [1/10], Step [900/1563], Loss: 1.5227
Epoch [1/10], Step [1000/1563], Loss: 1.5130
Epoch [1/10], Step [1100/1563], Loss: 1.4893
Epoch [1/10], Step [1200/1563], Loss: 1.4724
Epoch [1/10], Step [1300/1563], Loss: 1.4413
Epoch [1/10], Step [1400/1563], Loss: 1.4307
Epoch [1/10], Step [1500/1563], Loss: 1.4422
Epoch [1/10], Accuracy on test set: 49.64%
Epoch [2/10], Step [100/1563], Loss: 1.3840
Epoch [2/10], Step [200/1563], Loss: 1.3976
Epoch [2/10], Step [300/1563], Loss: 1.3334
Epoch [2/10], Step [400/1563], Loss: 1.3640
Epoch [2/10], Step [500/1563], Loss: 1.3506
Epoch [2/10], Step [600/1563], Loss: 1.3200
Epoch [2/10], Step [700/156

In [ ]:
from torchvision import datasets

def set_to_1000(_):
    return 1000

def set_to_1(_):    
    return 1
# OOD dataset (e.g. SVHN)
ood_dataset = datasets.SVHN(root="data", split="test", transform=validation_transform, download=True, target_transform=set_to_1000)
id_dataset = CIFAR10(root="./data", train=True, transform=train_transform, download=True, target_transform=set_to_1)
ood_loader = DataLoader(ood_dataset, batch_size=32, shuffle=False)
loader = DataLoader(ood_dataset + validation_dataset, batch_size=32, shuffle = True)

for images, labels in loader:
    print(len(labels))
    print(labels)  # Transformed labels (e.g., some are 1000)
    break


Using downloaded and verified file: data/test_32x32.mat
256
tensor([   2,    0,    0, 1000, 1000, 1000, 1000, 1000,    5,    6,    6, 1000,
           9, 1000,    1, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000,    1, 1000,    9,    9, 1000, 1000,    4, 1000,    8, 1000,    8,
        1000, 1000, 1000,    8,    5, 1000, 1000, 1000, 1000, 1000, 1000,    6,
        1000, 1000,    4, 1000, 1000, 1000,    1, 1000,    8,    0, 1000, 1000,
        1000, 1000, 1000,    1,    3, 1000,    8, 1000,    5,    0, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000,    3,    6,    1, 1000, 1000, 1000,
           0, 1000, 1000, 1000,    7, 1000, 1000,    6, 1000, 1000, 1000,    6,
        1000, 1000, 1000, 1000,    5, 1000, 1000, 1000, 1000, 1000,    7, 1000,
        1000, 1000,    6, 1000,    1, 1000, 1000, 1000, 1000,    0,    5, 1000,
        1000, 1000, 1000, 1000, 1000,    4,    1, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000,    6, 1000, 1000, 1000,    7

In [39]:
import torch
from pytorch_ood.utils import OODMetrics
from pytorch_ood.detector import OpenMax

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Wrap your trained model with OpenMax
detector = OpenMax(cnn_model.to(device))
cnn_model.eval()

# 2. Create OOD metrics collector
metrics = OODMetrics()

# 3. Fit OpenMax on in-distribution (ID) data
detector.fit(cifar_validation_loader)  # Make sure this loader has ID samples

# 4. Evaluate both ID and OOD
with torch.no_grad():
    # ---- OOD pass (label=True for each sample) ----
    for inputs, id in loader:
        inputs = inputs.to(device)
        metrics.update(detector(inputs), id)

# 5. Now compute the metrics (AUROC, etc.)
results = metrics.compute()
print(results)


ValueError: Data must contain ID and OOD samples.